## 타이타닉 생존자 예측
- 데이터 읽기
- 데이터 전처리
- 학습용 데이터 만들기
- 모델생성
- 모델검증
- 모델향상

In [149]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder #라벨링
from sklearn.model_selection import train_test_split # train/test데이터 나누기
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


In [2]:
raw_df = pd.read_csv('titanic_train.csv')

In [3]:
raw_df.info()
# Age도 확인해야하고
# Cabin이 204개네.. 687개가 null...?
# raw data는 건들지 말자 -> cl_df
# PassengerId 지워도 됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
cleansing_feature = ['PassengerId', 'Name']
cl_df = raw_df.drop(cleansing_feature, axis=1)

In [8]:
# Embarked 정리
cl_df.Embarked.fillna('N', inplace=True)

In [10]:
cl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [19]:
# cabin 정리
cl_df.Cabin.fillna('N', inplace=True)

In [21]:
cl_df['Cabin_tmp'] = cl_df.Cabin.str[:1]

In [17]:
# Age 정리
cl_df.Age.fillna(round(cl_df.Age.mean(),1), inplace=True) # Nan을 평균값으로 대체, 

In [18]:
cl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     891 non-null    object 
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [23]:
#
gender_encoder = LabelEncoder()
gender_encoder.fit(cl_df.Sex)

gender = gender_encoder.transform(cl_df.Sex) # 0과 1로 라벨링

In [24]:
gender

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,

In [25]:
cl_df['gender'] = gender # 대입

In [26]:
cl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   891 non-null    int64  
 1   Pclass     891 non-null    int64  
 2   Sex        891 non-null    object 
 3   Age        891 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Cabin      891 non-null    object 
 9   Embarked   891 non-null    object 
 10  Cabin_tmp  891 non-null    object 
 11  gender     891 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
Cabin_encoder = LabelEncoder()
Cabin_encoder.fit(cl_df.Cabin_tmp)
Cabin_class = Cabin_encoder.transform(cl_df.Cabin_tmp)

Embarked_encoder = LabelEncoder()
Embarked_encoder.fit(cl_df.Embarked)
Embarked_class = Embarked_encoder.transform(cl_df.Embarked)

In [28]:
cl_df['Cabin_class'] = Cabin_class
cl_df['Embarked_class'] = Embarked_class

In [31]:
cl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Survived        891 non-null    int64  
 1   Pclass          891 non-null    int64  
 2   Sex             891 non-null    object 
 3   Age             891 non-null    float64
 4   SibSp           891 non-null    int64  
 5   Parch           891 non-null    int64  
 6   Ticket          891 non-null    object 
 7   Fare            891 non-null    float64
 8   Cabin           891 non-null    object 
 9   Embarked        891 non-null    object 
 10  Cabin_tmp       891 non-null    object 
 11  gender          891 non-null    int64  
 12  Cabin_class     891 non-null    int64  
 13  Embarked_class  891 non-null    int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 97.6+ KB


## 최종 정리

In [33]:
feature_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'gender', 'Cabin_class', 'Embarked_class']
y_col = ['Survived']

X = cl_df[feature_cols]
y = cl_df[y_col]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [37]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [38]:
tree_pred = tree_model.predict(X_test)

In [39]:
accuracy_score(y_test, tree_pred)

0.7686567164179104

In [41]:
svc_model = SVC()
svc_model.fit(X_train, y_train)
svc_pred = svc_model.predict(X_test)
accuracy_score(y_test, svc_pred)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7126865671641791

In [43]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
accuracy_score(y_test, rf_pred)

<ipython-input-43-6fc40f921e82>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


0.7649253731343284

In [50]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
accuracy_score(y_test, lr_pred)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7723880597014925

In [51]:
kn_model = KNeighborsClassifier()
kn_model.fit(X_train, y_train)
kn_pred = kn_model.predict(X_test)
accuracy_score(y_test, kn_pred)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7574626865671642

In [58]:
lr_model.predict(np.array(X_test.iloc[0,:]).reshape(1,-1))

array([0])

In [59]:
lr_model.predict_proba(np.array(X_test.iloc[0,:]).reshape(1,-1))

array([[0.86501749, 0.13498251]])

In [52]:
y_test

,Survived
725,0
410,0
876,0
621,1
381,1
...,...
378,0
697,1
351,0
138,0


## HARD VOTING
- 각 분류기마다 모델생성
- 분류기부터 예측결과 취합
- 분류기가 예측한 투표 결과를 다수결로 하여 예측결과 산정
- accuracy_score(y_test,예측결과)을 이용하여 결과값 산출

In [120]:
v1 = RandomForestClassifier()
v2 = LogisticRegression()
v3 = KNeighborsClassifier()

In [121]:
v1.fit(X_train, y_train)
v2.fit(X_train, y_train)
v3.fit(X_train, y_train)

<ipython-input-121-7ad6bfd4533f>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  v1.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [122]:
v1_pred = v1.predict(X_test)
v2_pred = v2.predict(X_test)
v3_pred = v3.predict(X_test)

In [123]:
v1_pred

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0])

In [124]:
#result = sum(v1_pred.reshape(1,-1),(sum(v2_pred.reshape(1,-1), v3_pred.reshape(1,-1))))
result = v1_pred + v2_pred + v3_pred
hv = np.where(result >=2, 1, 0)

In [125]:
accuracy_score(y_test, hv)

0.753731343283582

### SOFT VOTING
- 각 분류기마다 모델생성
- 분류기부터 예측결과 취합(확률값)
- 분류기가 예측한 투표 결과를 확률의 기대값으로 예측결과 산정
- accuracy_score(y_test,예측결과)을 이용하여 결과값 산출

In [126]:
v1 = RandomForestClassifier()
v2 = LogisticRegression()
v3 = KNeighborsClassifier()

v1.fit(X_train, y_train)
v2.fit(X_train, y_train)
v3.fit(X_train, y_train)

<ipython-input-126-dcb2e9716228>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  v1.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [127]:
v1_pred = v1.predict_proba(X_test)
v2_pred = v2.predict_proba(X_test)
v3_pred = v3.predict_proba(X_test)

In [128]:
v1_pred

array([[1.        , 0.        ],
       [0.93092649, 0.06907351],
       [1.        , 0.        ],
       [0.69      , 0.31      ],
       [0.29566667, 0.70433333],
       [0.39      , 0.61      ],
       [0.93092649, 0.06907351],
       [1.        , 0.        ],
       [0.42533333, 0.57466667],
       [0.22      , 0.78      ],
       [0.98      , 0.02      ],
       [0.93      , 0.07      ],
       [0.88      , 0.12      ],
       [0.45      , 0.55      ],
       [0.04090909, 0.95909091],
       [0.05      , 0.95      ],
       [0.01      , 0.99      ],
       [0.9325    , 0.0675    ],
       [0.4       , 0.6       ],
       [0.39      , 0.61      ],
       [1.        , 0.        ],
       [0.04      , 0.96      ],
       [0.03      , 0.97      ],
       [1.        , 0.        ],
       [0.68368458, 0.31631542],
       [0.18      , 0.82      ],
       [0.12266667, 0.87733333],
       [0.70420696, 0.29579304],
       [0.1895    , 0.8105    ],
       [0.49166667, 0.50833333],
       [0.

In [129]:
(v1_pred + v2_pred + v3_pred).shape

(268, 2)

In [131]:
# 죽을 확률에서 살 확률 빼기 
sv = np.where((v1_pred + v2_pred + v3_pred)[:,0] - (v1_pred + v2_pred + v3_pred)[:,1] > 0
        ,0
        ,1)

In [132]:
accuracy_score(y_test, sv)

0.7947761194029851

In [167]:
c1 = RandomForestClassifier()
c2 = LogisticRegression()
c3 = KNeighborsClassifier()
c4 = RandomForestClassifier()
c5 = KNeighborsClassifier()

In [168]:
c1.fit(X_train, y_train)
c2.fit(X_train, y_train)
c3.fit(X_train, y_train)
c4.fit(X_train, y_train)
c5.fit(X_train, y_train)

<ipython-input-168-81839fb8a08d>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  c1.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
<ipython-input-168-81839fb8a08d>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  c4.fit(X_train, y_train)
/opt/anaconda3/envs/ml/l

KNeighborsClassifier()

In [169]:
c1_pred = c1.predict(X_test)
c2_pred = c2.predict(X_test)
c3_pred = c3.predict(X_test)
c4_pred = c4.predict(X_test)
c5_pred = c5.predict(X_test)

In [170]:
result = c1_pred + c2_pred + c3_pred + c4_pred + c5_pred
hv = np.where(result >=3, 1, 0)

In [171]:
accuracy_score(y_test, hv)

0.7686567164179104

In [172]:
c1_pred = c1.predict_proba(X_test)
c2_pred = c2.predict_proba(X_test)
c3_pred = c3.predict_proba(X_test)
c4_pred = c4.predict_proba(X_test)
c5_pred = c5.predict_proba(X_test)

In [173]:
sv = np.where((c1_pred + c2_pred + c3_pred + c4_pred + c5_pred)[:,0] - (c1_pred + c2_pred + c3_pred + c4_pred + c5_pred)[:,1] > 0
        ,0
        ,1)

In [174]:
accuracy_score(y_test, sv)

0.7835820895522388

In [ ]:
#######################################

In [189]:
rf = RandomForestClassifier()
lr = LogisticRegression()
kn = KNeighborsClassifier()
meta = DecisionTreeClassifier()

In [193]:
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)
kn.fit(X_train, y_train)
rf_result = rf.predict(X_train)
lr_result = lr.predict(X_train)
kn_result = kn.predict(X_train)

<ipython-input-193-1d2d86e910a0>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [194]:
temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['lr_result'] = lr_result
temp['kn_result'] = kn_result

In [195]:
temp

,rf_result,lr_result,kn_result
0,1,1,0
1,0,0,0
2,1,1,1
3,0,0,0
4,1,0,1
...,...,...,...
618,1,0,1
619,0,0,0
620,0,0,0
621,1,1,0


In [196]:
meta.fit(temp.values, y_train.values)

DecisionTreeClassifier()

In [ ]:
meta.predict(temp.values)

In [ ]:
#########

In [198]:
rf_result = rf.predict(X_test)
lr_result = lr.predict(X_test)
kn_result = kn.predict(X_test)

In [199]:
temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['lr_result'] = lr_result
temp['kn_result'] = kn_result

In [200]:
st_result = meta.predict(temp.values)

In [220]:
accuracy_score(y_test,st_result)

0.7723880597014925

In [219]:
#########################################

In [209]:
rf = RandomForestClassifier()
tree = DecisionTreeClassifier()
kn = KNeighborsClassifier()
meta = LogisticRegression()

In [210]:
rf.fit(X_train, y_train)
tree.fit(X_train, y_train)
kn.fit(X_train, y_train)
rf_result = rf.predict(X_train)
tree_result = tree.predict(X_train)
kn_result = kn.predict(X_train)

<ipython-input-210-5c723434eaec>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [217]:
temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['tree_result'] = tree_result
temp['kn_result'] = kn_result

In [218]:
temp

,rf_result,tree_result,kn_result
0,1,1,0
1,0,0,0
2,1,1,1
3,0,0,0
4,1,1,1
...,...,...,...
618,1,1,1
619,0,0,0
620,0,0,0
621,1,1,0


In [213]:
meta.fit(temp.values, y_train.values)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [214]:
meta.predict(temp.values)

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,

In [221]:
rf_result = rf.predict(X_test)
tree_result = tree.predict(X_test)
kn_result = kn.predict(X_test)

temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['tree_result'] = tree_result
temp['kn_result'] = kn_result

st_result = meta.predict(temp.values)
accuracy_score(y_test,st_result)

0.7873134328358209

In [ ]:
############################ proba

In [222]:
rf.fit(X_train, y_train)
tree.fit(X_train, y_train)
kn.fit(X_train, y_train)
rf_result = rf.predict_proba(X_train)[:, 0] -  rf.predict_proba(X_train)[:, 1]
tree_result = tree.predict_proba(X_train)[:, 0] - tree.predict_proba(X_train)[:, 1]
kn_result = kn.predict_proba(X_train)[:, 0] - kn.predict_proba(X_train)[:, 1]

<ipython-input-222-befc97e1d102>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [223]:
temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['tree_result'] = tree_result
temp['kn_result'] = kn_result

In [224]:
temp

,rf_result,tree_result,kn_result
0,-0.840000,-1.000000,0.2
1,0.940000,1.000000,1.0
2,-0.980000,-1.000000,-0.6
3,1.000000,1.000000,1.0
4,-0.820000,-1.000000,-1.0
...,...,...,...
618,-0.299500,-0.333333,-0.2
619,0.815187,0.800000,0.6
620,0.850069,0.862069,0.6
621,-0.941846,-1.000000,0.2


In [225]:
meta.fit(temp.values, y_train.values)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [226]:
rf_result = rf.predict_proba(X_test)[:, 0] -  rf.predict_proba(X_test)[:, 1]
tree_result = tree.predict_proba(X_test)[:, 0] - tree.predict_proba(X_test)[:, 1]
kn_result = kn.predict_proba(X_test)[:, 0] - kn.predict_proba(X_test)[:, 1]


In [227]:
temp = pd.DataFrame()
temp['rf_result'] = rf_result
temp['tree_result'] = tree_result
temp['kn_result'] = kn_result

temp

,rf_result,tree_result,kn_result
0,1.000000,1.000000,1.0
1,0.850069,0.862069,0.6
2,1.000000,1.000000,1.0
3,0.420000,-1.000000,0.2
4,-0.640667,-1.000000,-1.0
...,...,...,...
263,-0.720000,-1.000000,-0.2
264,-0.516619,-0.500000,-0.6
265,-0.017686,0.000000,-0.2
266,0.975000,1.000000,1.0


In [228]:
st_result = meta.predict(temp.values)
accuracy_score(y_test, st_result)

0.7686567164179104